# Description

This notebook is a template notebook that is intended to be run across different parameters.

Based on the settings below, it loads an input file with paragraph pairs (original and revised) and uses the LLM-as-a-Judge approach to evaluate the quality of the paragraphs.

# Modules

In [1]:
import pandas as pd
from IPython.display import display
from langchain.cache import SQLiteCache
from langchain.globals import set_llm_cache
from proj import conf
from proj.utils import llm_pairwise

# Settings/paths

In [2]:
# Input manuscript
REPO = None

INPUT_FILE = None
OUTPUT_FILE = None

# Model and its parameters
LLM_JUDGE = None
TEMPERATURE = None
MAX_TOKENS = 2000
SEED_INIT = 0

# Evaluation parameters
N_REPS = None
THROW_IF_FAILED = False

In [3]:
# Parameters
REPO = "pivlab/manubot-ai-editor-code-test-biochatter-manuscript"
INPUT_FILE = "/home/miltondp/projects/others/manubot/manubot-ai-editor-code/base/results/paragraph_match/biochatter-manuscript--gpt-3.5-turbo.pkl"
OUTPUT_FILE = "/home/miltondp/projects/others/manubot/manubot-ai-editor-code/base/results/llm_pairwise/biochatter-manuscript--gpt-3.5-turbo--openai_gpt-4-turbo-preview.pkl"
LLM_JUDGE = "openai:gpt-4-turbo-preview"
TEMPERATURE = 0.5
MAX_TOKENS = 2000
SEED_INIT = 0
N_REPS = 5


In [4]:
conf.common.LLM_CACHE_DIR.mkdir(parents=True, exist_ok=True)
display(conf.common.LLM_CACHE_DIR)

PosixPath('/home/miltondp/projects/others/manubot/manubot-ai-editor-code/base/results/llm_cache')

# Set default LangChain cache file

In [5]:
default_cache_file = conf.common.LLM_CACHE_DIR / "default.db"
display(default_cache_file)
set_llm_cache(SQLiteCache(database_path=str(default_cache_file)))

PosixPath('/home/miltondp/projects/others/manubot/manubot-ai-editor-code/base/results/llm_cache/default.db')

# Load paragraphs

In [6]:
df = pd.read_pickle(INPUT_FILE)

In [7]:
df.shape

(37, 3)

In [8]:
df.head()

,section,original,modified
0,abstract,Current-generation Large Language Models (LLMs...,Large Language Models (LLMs) have generated si...
1,introduction,"Despite technological advances, understanding ...","Despite technological advances, understanding ..."
2,introduction,Large Language Models (LLMs) of the current ge...,The latest generation of Large Language Models...
3,introduction,Computational biomedicine involves many tasks ...,Computational biomedicine encompasses various ...
4,results,The framework is designed to be modular: any o...,"The framework is designed to be modular, allow..."


In [9]:
df.iloc[0]["original"]

'Current-generation Large Language Models (LLMs) have stirred enormous interest in recent months, yielding great potential for accessibility and automation, while simultaneously posing significant challenges and risk of misuse. To facilitate interfacing with LLMs in the biomedical space, while at the same time safeguarding their functionalities through sensible constraints, we propose a dedicated, open-source framework: BioChatter. Based on open-source software packages, we synergise the many functionalities that are currently developing around LLMs, such as knowledge integration / retrieval-augmented generation, model chaining, and benchmarking, resulting in an easy-to-use and inclusive framework for application in many use cases of biomedicine. We focus on robust and user-friendly implementation, including ways to deploy privacy-preserving local open-source LLMs. We demonstrate use cases via two multi-purpose web apps ([https://chat.biocypher.org](https://chat.biocypher.org)), and pr

In [10]:
df.iloc[0]["modified"]

'Large Language Models (LLMs) have generated significant interest due to their potential for accessibility and automation in various fields, including biomedicine. However, they also present challenges and risks of misuse. In this paper, we address the need for a framework to interface with LLMs in the biomedical domain while ensuring their safe and effective use. To meet this need, we introduce BioChatter, an open-source framework that integrates various functionalities of LLMs, such as knowledge integration, retrieval-augmented generation, model chaining, and benchmarking. By leveraging open-source software packages, we have developed a user-friendly and versatile platform that can be applied across a range of biomedicine use cases. Our focus is on implementing robust and privacy-preserving local open-source LLMs. We showcase the utility of BioChatter through two multi-purpose web apps available at [https://chat.biocypher.org](https://chat.biocypher.org) and provide comprehensive doc

# Test run

In [11]:
t_json = llm_pairwise(
    df.iloc[0]["original"],
    df.iloc[0]["modified"],
    df.iloc[0]["section"],
    model_name=LLM_JUDGE,
    model_params={
        "temperature": TEMPERATURE,
        "max_tokens": MAX_TOKENS,
        "model_kwargs": {
            "seed": SEED_INIT,
        },
    },
    verbose=True,
)



> Entering new LLMChain chain...
Prompt after formatting:
System: You are an expert copyeditor with ample experience in scientific writing. You are assessing the quality of two versions of a paragraph from the Abstract of a scientific article.
Human: Evaluate the quality of the following paragraph by writing a list with positive (if any) and/or negative (if any) aspects on the following areas: 1) has a clear sentence structure, 2) is easy to follow, 3) is correct in grammar, 4) has no spelling errors.

Paragraph A: Current-generation Large Language Models (LLMs) have stirred enormous interest in recent months, yielding great potential for accessibility and automation, while simultaneously posing significant challenges and risk of misuse. To facilitate interfacing with LLMs in the biomedical space, while at the same time safeguarding their functionalities through sensible constraints, we propose a dedicated, open-source framework: BioChatter. Based on open-source software packages, we

In [12]:
t_json

{'best': 'Paragraph 1',
 'rationale': 'Paragraph 1 is selected as the better of the two based on its clearer sentence structure and enhanced ease of following. While both paragraphs are grammatically correct and free from spelling errors, Paragraph 1 presents its information in a more accessible manner, with a logical flow that gently guides the reader from the problem statement to the proposed solution without overloading them with technical jargon. Although both paragraphs utilize technical terms without extensive explanation, Paragraph 1 does so in a slightly less dense manner, making it marginally easier for readers to grasp the core concepts. The direct web link inclusion is a common negative aspect in both paragraphs, but it does not significantly detract from the overall quality of Paragraph 1. Therefore, based on the criteria of clear sentence structure, ease of following, grammatical correctness, and absence of spelling errors, Paragraph 1 is deemed superior.'}

In [13]:
type(t_json)

dict

# Run

Since models are stochastic, we run the pairwise comparison many times.

Here I use a cache to avoid hitting an external API multiple times.

In [14]:
results = []

In [15]:
for rep_idx in range(N_REPS):
    # we cache prompt/results by repetition
    output_cache_file = conf.common.LLM_CACHE_DIR / f"rep{rep_idx}.db"
    set_llm_cache(SQLiteCache(database_path=str(output_cache_file)))

    print(f"{str(rep_idx).zfill(2)} ({output_cache_file.name}): ", end="", flush=True)

    for par_idx, par in df.iterrows():
        print(".", end="", flush=True)

        res = llm_pairwise(
            par["original"],
            par["modified"],
            par["section"],
            model_name=LLM_JUDGE,
            model_params={
                "temperature": TEMPERATURE,
                "max_tokens": MAX_TOKENS,
                "model_kwargs": {
                    "seed": SEED_INIT + rep_idx,
                },
            },
            throw_if_failed=THROW_IF_FAILED,
            verbose=False,
        )

        results.append(
            {
                "rep_index": rep_idx,
                "paragraph_index": par_idx,
                "paragraph_section": par["section"],
                "winner": res["best"],
                "rationale": res["rationale"],
            }
        )

    print(flush=True)

00 (rep0.db): 

.

.

.

.

...

.

.

.

.

.

# Process results

In [16]:
winner_matchings = {
    "Paragraph A": "-1",  # Original
    "Paragraph 1": "1",  # Modified
    "tie": "0",
}

In [17]:
df_results = pd.DataFrame(results)

In [18]:
df_results.shape

(185, 5)

In [19]:
df_results.head()

,rep_index,paragraph_index,paragraph_section,winner,rationale
0,0,0,abstract,Paragraph 1,When comparing the two paragraphs across the e...
1,0,1,introduction,Paragraph 1,Both paragraphs address similar content with c...
2,0,2,introduction,Paragraph 1,Both paragraphs aim to discuss the capabilitie...
3,0,3,introduction,tie,Both Paragraph A and Paragraph 1 are well-cons...
4,0,4,results,tie,Both paragraphs effectively communicate the fu...


In [20]:
df_results["winner"].value_counts()

winner
Paragraph 1    95
tie            90
Name: count, dtype: int64

In [21]:
df_results = df_results[df_results["winner"].isin(winner_matchings.keys())]

In [22]:
df_results.shape

(185, 5)

In [23]:
df_results = df_results.assign(
    winner_score=df_results["winner"].replace(winner_matchings).apply(float)
)

In [24]:
df_results.shape

(185, 6)

In [25]:
df_results.head()

,rep_index,paragraph_index,paragraph_section,winner,rationale,winner_score
0,0,0,abstract,Paragraph 1,When comparing the two paragraphs across the e...,1.0
1,0,1,introduction,Paragraph 1,Both paragraphs address similar content with c...,1.0
2,0,2,introduction,Paragraph 1,Both paragraphs aim to discuss the capabilitie...,1.0
3,0,3,introduction,tie,Both Paragraph A and Paragraph 1 are well-cons...,0.0
4,0,4,results,tie,Both paragraphs effectively communicate the fu...,0.0


In [26]:
df_results.dtypes

rep_index              int64
paragraph_index        int64
paragraph_section     object
winner                object
rationale             object
winner_score         float64
dtype: object

In [27]:
df_results.groupby("paragraph_section")["winner_score"].mean()

paragraph_section
abstract        1.000000
discussion      0.514286
introduction    0.733333
methods         0.275000
results         0.780000
Name: winner_score, dtype: float64

# Save

In [28]:
df_results.to_pickle(OUTPUT_FILE)